In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms
#import czifile
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time
import os
import sys
import torchvision
from torch.utils.data import Dataset, DataLoader
# from aicspylibczi import CziFile
# import czifile
import numpy as np
import matplotlib.pyplot as plt
import cv2
import imageio
import time
#from cellpose import io, models

In [ ]:
from google.cloud import storage

# Create a storage client.
storage_client = storage.Client(project='jal2340-applied-cv-s24')

# Specify the name of your GCS bucket and the object (file) to download.
bucket_name = '3d-dicty-data'

bucket = storage_client.bucket(bucket_name)

In [ ]:
# exp is experiment/ collection dates
exp_dates = ['2023-01-30', '2023-10-25']
blobs = {
    exp_dates[0] : {"MIP" : [], "raw": [], "processed":[]},
    exp_dates[1] : dict(raw=[], MIP=[], smiley=[]),
}
for blob in bucket.list_blobs():
  if(exp_dates[0] in blob.name):
    if('MIP.czi' in blob.name):
      blobs[exp_dates[0]]["MIP"].append(blob)
    elif("processed.czi" in blob.name):
      blobs[exp_dates[0]]["processed"].append(blob)
    else:
      blobs[exp_dates[0]]["raw"].append(blob)
  elif(exp_dates[1] in blob.name):
    if('MIP.czi' in blob.name):
      blobs[exp_dates[1]]["MIP"].append(blob)
    elif("smiley.czi" in blob.name):
      blobs[exp_dates[1]]["smiley"].append(blob)
    else:
      blobs[exp_dates[1]]["raw"].append(blob)

In [ ]:
blob = blobs[exp_dates[0]]['processed'][5]
blob.size / (1024 * 1024 * 1024) # size in GB

In [ ]:
chunk_size=10*1024*1024

In [ ]:
def get_image(vid_index, date=0, group='processed', blobs=blobs):
  blob = blobs[exp_dates[date]][group][vid_index]
  fname = blob.name.split('/')[-1] # ignore date/fname
  print(f"Loading {fname}")
  blob.download_to_filename(fname)
  return(CziFile(fname))

In [ ]:
img = get_image(5, 0, 'processed')
# TODO: Understand each dimension in the shape
# shape is Channels x num_slices x H x W.
# S is "Scene" - different POV or regions
# not a video ??
img.get_dims_shape()

In [ ]:
img_input = img[0, 50, :, :]

In [ ]:
masks, flows, styles = models.CellposeModel(model_type='cyto3').eval(img_input,
                            diameter=None, channels=[0,0])

In [ ]:
plt.imshow(img[0, 50, :, :])

In [ ]:
masks.unique()

In [ ]:
def get_zslice(img, z, channel=0):
  return(img[channel, z, :, :])

In [ ]:
plt.imshow(get_zslice(img, 50))

NameError: name 'img' is not defined

In [118]:
def get_bounding_boxes(a, padding = 0):
      num_rectangles = np.max(a)
      rectangle_coords = []
      for i in np.arange(num_rectangles) + 1:
        values = np.where(a == i)
        x = values[0][0]
        y = values[1][0]
        if (x!=0):
          if (padding > x):
            x = padding - x
          else:
            width = np.abs(values[0][-1] - x) + padding*2 + 1

        else:
          x = 0
          width = np.abs(values[0][-1] - x) + padding + 1



        if (y!=0):
          print(padding, "y is", y)
          if (padding > y):
            y = padding - y
          else:
            height = np.abs(values[1][-1] - y) + padding*2 +1
        else:
          y=0
          height = np.abs(values[1][-1] - y)  + padding +1

        #check that height doesnt go over the page
        if (a.shape[1]+1 < height+y):
          height  = height - (((height+y)-a.shape[1])-1)


        rectangle_coords.append((x,  width))

      return rectangle_coords





#x,y,w,h

In [158]:
def get_bounding_boxes(a, padding = 0):
  num_rectangles = np.max(a)
  rectangle_coords = []
  for i in np.arange(num_rectangles) + 1:
        values = np.where(a == i)
        x = values[0][0]
        y = values[1][0]

        #first we pick the right x value b y subtarcting padding. If the new x value is smaller than 0 then x is just 0
        x = x - padding
        width = np.abs(values[0][-1] - x) + padding + 1
        if (x < 0):
          #subtract the amounts we are removing from
          width = width - abs(x)
          x = 0

        #now we want to cut off the width if we go over
        if (x+width > a.shape[0]):
          width = width - ((x+width)-a.shape[0] )


        #repeat for y

        y = y - padding
        height = np.abs(values[1][-1] - y) + padding + 1
        if (y < 0):
          height = height - abs(y)
          y = 0
        if (y+height > a.shape[1]):
          height = height - ((y+height)-a.shape[1] )

        rectangle_coords.append((x,  width, y, height))

  return rectangle_coords








In [167]:
a = [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2],
     [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 2, 2, 2, 2, 2, 2],
     [1, 1, 1, 1, 0, 0, 0, 5, 0, 0, 0, 3, 3, 0, 0, 0, 2, 2, 2, 2, 2, 2],
     [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2],
     [1, 1, 1, 1, 0, 0, 0, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0]]

a = np.array(a)
get_bounding_boxes(a, padding = 15)

[(0, 6, 0, 19),
 (0, 6, 0, 22),
 (0, 6, 0, 22),
 (0, 6, 0, 22),
 (0, 6, 0, 22),
 (0, 6, 5, 17)]

In [ ]:
def get_bounding_cube(list_of_a, padding=0):
  for a in list_of_a:
    index_coords = get_bounding_boxes(a)
    #this returns a list of [(x,y,width,height),....] for each index


